# Colab-Backend for [KOI](https://github.com/nousr/koi)
---

This notebook serves as one of the many ways you connect `koi` to a GPU backend!

I also hope that it will serve as a good "getting started" guide and walk you through all the steps necessary to get everything up and running!


### Notebook & Plug-In by [nousr](https://twitter.com/nousr_)

---

*StableDiffusion is a model created by CompVis in conjunction with [StabilityAI](stability.ai). By using this notebook you are also agreeing to any binding agreements that are associated with the StableDiffusion-V1 model.*

## Dependencies

First we need to install a few things...

1. We install the koi package to ensure we have the proper packages
2. We install `diffusers` from source to get the latest *img2img* pipeline
3. Finally we install `ngrok` and `flask-ngrok` *(**note:** this are not necessary if you are running the server locally)*


`Ngrok` & `Flask` is what makes it possible to use google colab as our gpu backend. In short, flask will handle our our server and ngrok will provide us a public IP that we can use to talk to from our local machine. 


In [ ]:
!git clone https://github.com/nousr/koi.git && pip install -e koi
!pip install git+https://github.com/huggingface/diffusers.git
!pip install ngrok
!pip install flask-ngrok
!sudo apt install net-tools 

# ⏰ NOTE ⏰

Before you continue you need to refresh the notebook, otherwise you will get some mysterious errors!

You can do this by going to the top menu bar in colab and navigating to `Runtime` > `Restart Runtime`. After it refreshes you can continue!

> *note:* you do not need to re-run the first cell after restarting the runtime

(thanks to @thefacesblur on twitter for helping me debug this)

## Setup

Before we continue you must login to huggingface to use stable diffusion through the `diffusers` module.

To do this you will need to do two things:

1. **Important!** Accept the OpenRAIL license for stable diffusion here https://huggingface.co/CompVis/stable-diffusion-v1-4 
2. Go to https://huggingface.co/settings/tokens and generate a new token to use.

> ***NOTE:*** If you do not complete the second step, or do not do so without generating the token on the same account you accepted the license you will run into errors below.

In [ ]:
!huggingface-cli login # make sure you hit enter after pasting your token

## The backend

Now we can finally setup our backend server. For now we will keep it simple and have one API endpoint. 

> ⏰ **NOTE**! ⏰
>
> If you get an error saying the `transformers` library is not installed you probably need to restart the runtime to refresh the environment. 
> 
> You can do this by navigating to `Runtime` > `Restart Runtime`. After it refreshes you can try re-running this cell and it should work. *(You do ***not*** need to re-execute the other cells)*

---

In the future setting up the colab-backend will hopefully be as smple as just doing something like...

```python
from koi import colab_server

run_server()
```

...but for now we will do it explicitly 🙂



In [ ]:
import torch
from flask import Flask, Response, request, send_file
from PIL import Image
from io import BytesIO
from torch import autocast
from diffusers import StableDiffusionImg2ImgPipeline
from click import secho
from zipfile import ZipFile

# the following line is specific to remote environments (like google colab)
from flask_ngrok import run_with_ngrok

# Load the model for use (this may take a minute or two...or three)
secho("Loading Model...", fg="yellow")

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4", use_auth_token=True
).to("cuda")

secho("Finished!", fg="green")

# Start setting up flask

app = Flask(__name__)

# Define a function to help us "control the randomness"

def seed_everything(seed: int):
    import random, os

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

def get_name(prompt, seed):
  return f'{prompt}-{seed}'

# Define one endpoint "/api/img2img" for us to communicate with
@app.route("/api/img2img", methods=["POST"])
def img2img():
    global pipe

    r = request
    headers = r.headers

    data = r.data
    buff = BytesIO(data)
    img = Image.open(buff).convert("RGB")

    seed = int(headers["seed"])
    prompt = headers['prompt']


    print(r.headers)

    zip_stream = BytesIO()
    with ZipFile(zip_stream, 'w') as zf:

        for index in range(int(headers['variations'])):
            variation_seed = seed + index
            seed_everything(variation_seed)
        
            with autocast("cuda"):
                return_image = pipe(
                    init_image=img,
                    prompt=prompt,
                    strength=float(headers["sketch_strength"]),
                    guidance_scale=float(headers["prompt_strength"]),
                    num_inference_steps=int(headers["steps"]),
                )["sample"][0]


            return_bytes = BytesIO()
            return_image.save(return_bytes, format="JPEG")

            return_bytes.seek(0)
            zf.writestr(get_name(prompt, variation_seed), return_bytes.read())

    zip_stream.seek(0)

    return send_file(zip_stream, mimetype="application/zip")

run_with_ngrok(app)
app.run()

# Plugging into Krita
(last step!)

At this point, if everything worked, you should see something like the following!
```terminal
Finished!
 * Serving Flask app '__main__'
 * Debug mode: off
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
 * Running on http://c44b-34-124-187-78.ngrok.io <----❗This is what we need❗
 * Traffic stats available on http://127.0.0.1:4040 
 ```

Everytime you run this notebook, ngrok will give you a new public-ip to use...in my case this was `http://c44b-34-124-187-78.ngrok.io`.

**IMPORTANT: To begin using koi you will paste this public ip into the `endpoint` field, along with our *api route*. Like so:**

`http://c44b-34-124-187-78.ngrok.io/api/img2img`

---

### Have fun! feel free to tweet me your creations--I'd love to see them :)

# Developers Only


If you need to re-start the flask server in the course of development, killing the cell where it is running won't be enough. Instead you will need to use the commands below to kill the current runtime (which will also kill the running flask server). You can then re-run the flask code again and changes will propagate without needing to reinstall dependencies, log back into huggingface or, download the model again.

In [ ]:
!sudo netstat -tulnp | grep :5000

In [ ]:
!sudo kill 434